<a href="https://colab.research.google.com/github/halen48/VisualG2C/blob/main/VisualG2C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [477]:
import re

In [478]:
nome_arq_c = "arquivo.c" #@param

#Programa

In [479]:
programa = """
algoritmo "Calculos de numeros"
// Função : Exibe informações como o maior, menor número, a média e a sua soma
// Autor : Guilherme Novaes
// Data : 11/03/2022
// Seção de Declarações
var
numeros: vetor [1..5] de inteiro
teste1,teste2,teste3, teste4 : vetor [1..3,1..4] de real
nome : caractere
max, min,i,soma:inteiro
media : real
inicio
// Seção de Comandos

escreva("Digite seu nome: ")
leia(nome)

max<-numeros[1];
para i de 1 ate 5 faca
  escreva (nome, ", digite o ", i,"o valor: ")
  leia(numeros[i])

  // soma dos valores digitados
  soma <- soma + numeros[i];
  //media entre os valores digitados
  media<- soma / 5;

  se (numeros[i] > max) entao
    max<- numeros[i];
  fimse

  min <- numeros[1];
  se (numeros[i] < min) entao
    min <- numeros[i];
  fimse
fimpara

escreval ("maior numero: ", max)
escreval ("menor numero: ",min)
escreval("Soma dos numeros digitados:",soma)
escreva ("Média dos numero digitados:", media)

fimalgoritmo
"""

#Parse

##Funções de Parse e Constantes

In [480]:
palavras_chave = {
    "inicio" : "fimalgoritmo",
    "se" : 'fimse',
    "para" : 'fimpara',
    "enquanto" : 'fimenquanto',
    "escolha" : "fimescolha"
}

In [481]:
palavras_chave_fechamento = {}
for palavra in palavras_chave:
  palavras_chave_fechamento[palavras_chave[palavra]] = palavra

In [482]:
tipos_variaveis = {
    'inteiro' : 'int',
    'real' : 'float',
    'caractere' : 'char',
    'logico' : 'bool',
}

In [483]:
mascaras_tipos_c = {
    'int' : '%d',
    'float' : '%f',
    'char' : '%s',
    'bool' : '%d',
}

```
Tamanho máximo da string que o visualG suporta
```

In [484]:
max_tam_string = 500 #@param {type:"integer"}

###Sub-parces

###Apelidos: Utilizados em um simples replace

In [485]:
apelidos = {
    '=' : '==',
    '<>' : '!=',
    '\\' : '/',
    'mod' : '%',
    'div' : '/',
    '<-' : '=',
    'limpatela' : '',
}

###Regex

In [486]:
vetor_mask = re.compile(r'(.*): *vetor *\[(.*)\] * de * (\w+)')
var_mask = re.compile(r'(.*) *: *(\w+)')
potencia_mask = re.compile(r'([^\^]+)\^([^\-\+\*\\]+)') #pow([0],[1])
parametros_mask = re.compile(r'\(([^\)]*)\)')
para_mask = re.compile(r"para *([^ ]+) *de *([^ ]+) *ate *([^ ]+) faca")
se_mask = re.compile(r"se *(.*) *entao")
string_mask = re.compile(r'\"([^\"]+)\"')

###Utilidades

In [487]:
def eh_funcao(valor, linha):
  return (valor == linha[:len(valor)])

In [488]:
def nome_variavel(nome):
  idx = nome.find('[')
  if(idx > 0):
    return nome[:idx]
  return nome

###Casos especiais

In [489]:
def parse_inicio(linha, n_vars, t_vars):
  linha = linha.strip()
  if(not eh_funcao('inicio', linha)):
    return None
  
  ret = """
#include <stdio.h>
#include <stdbool.h>
#include <string.h>

int main() {
"""

  for tipo in t_vars:
    ret += '\t'+tipo + ' '
    for i in range(0,len(t_vars[tipo]), 5):
       ret += ', '.join(t_vars[tipo][i:i+5])
    ret += ';\n'
  
  ret += '\n'.join(['\tmemset(&%s, 0, sizeof(%s));'%(nome,nome) for nome in n_vars])

  return ret

In [490]:
def parse_fimalgoritmo(linha, _, __):
  linha = linha.strip()
  if(not eh_funcao('fimalgoritmo', linha)):
    return None
  return """
  return 0;
}
"""

In [491]:
def parse_escreva(linha, n_vars, _):
  linha = linha.strip()
  if(not eh_funcao('escreva', linha)):
    return None
  pula_linha = ('escreval' == linha[:len('escreval')])
  printf_params = ""
  vars = []

  try:
    params = parametros_mask.findall(linha)[0]
  except:
    raise Exception("Erro de sintaxe em <%s>"%linha)
  #primeiro isola as strings
  strs = string_mask.findall(params)
  for i,s in enumerate(strs):
    params = params.replace(s,'{%d}'%i)
  #agora realiza as outras operacoes
  for param in params.split(','):
    param = param.strip()
    nome = nome_variavel(param)
    print(param, nome)
    if(nome in n_vars):
      printf_params += mascaras_tipos_c[n_vars[nome]]
      vars.append(param)
    elif(param[0] == '"'):
      printf_params += strs[int(param[2:-2])]
    else:
      print(linha)
      raise Exception("Variavel %s não existe!!"%param)
  
  if(pula_linha):
    printf_params += '\\n'

  printf_params += '"'
  if(vars):
    printf_params += ', '+', '.join(vars)
  
  return ('printf("%s);'%printf_params)

In [492]:
parse_escreval = parse_escreva

In [493]:
def parse_leia(linha, n_vars, _):
  linha = linha.strip()
  if(not eh_funcao('leia', linha)):
    return None
  
  scanf_params = ""
  vars = []
  try:
    params = parametros_mask.findall(linha)[0]
  except:
    raise Exception("Erro de sintaxe em <%s>"%linha)

  for param in params.split(','):
    param = param.strip()
    nome = nome_variavel(param)
    if(nome in n_vars):
      scanf_params += mascaras_tipos_c[n_vars[nome]]
      
      if(n_vars[nome] == 'char'):
        vars.append(param)
      else:
        vars.append("&"+param)
    else:
      raise Exception("Variavel %s não existe!!"%param)
  

  scanf_params += '"'
  if(vars):
    scanf_params += ', '+', '.join(vars)
  
  return ('scanf("%s);'%scanf_params)

In [494]:
def parse_para(linha, n_vars, _):
  if(not eh_funcao('para', linha)):
    return None
  
  try:
    var, inicio, fim = para_mask.findall(linha)[0]
  except:
    raise Exception("Erro de sintaxe em <%s>"%linha)

  if(not nome_variavel(var) in n_vars):
    raise Exception ("<%s> não declarada!!"%var)
  if(inicio > fim): #decrementa
    return ('for(%s = %s; %s >= %s; %s--) {'%(var, inicio, var, fim, var))
  else:
    return ('for(%s = %s; %s <= %s; %s++) {'%(var, inicio, var, fim, var))

In [495]:
def parse_se(linha, n_vars, _):
  if(not eh_funcao('se', linha)):
    return None
  
  try:
    exp = se_mask.findall(linha)[0]
  except:
    raise Exception("Erro de sintaxe em <%s>"%linha)

  return ('if (%s) {'%(exp))

###Outras funções

In [496]:
def limpa_comentario(codigo):
  codigo_final = ""

  ignora = False
  for idx in range(len(codigo)):
    if(ignora):
      if(codigo[idx] == '\n'):
        ignora = False
    else:
      if(codigo[idx:idx+2] == '//'):
          ignora = True
      else:
        codigo_final += codigo[idx]

  return codigo_final.replace('\t',' ')

In [497]:
def variaveis_globais(codigo):
  #=====================================================================
  def var_dict(v):
    if(not v in t_vars):
        t_vars[v] = []
  #=====================================================================
  t_vars = {} #tipo : nome
  n_vars = {} #nome : tipo
  escopo = codigo[:codigo.find('inicio')]
  for nomes_var, tipo in var_mask.findall(escopo):
  # ============ Variaveis simples =====================================
    nomes_var, tipo = nomes_var.lower(), tipo.lower()
    if(tipo == 'vetor'):  
      continue #Tradados depois...
    var_dict(tipos_variaveis[tipo])
    #Tipo caractere no visual G é uma string
    if(tipo == 'caractere'):
      t_vars[tipos_variaveis[tipo]] += [var.strip()+'[%d]'%max_tam_string for var in nomes_var.split(',')]
    else:
      t_vars[tipos_variaveis[tipo]] += [var.strip() for var in nomes_var.split(',')]
  # ============= Vetores ==============================================
  for nomes_var, dimensoes, tipo in vetor_mask.findall(escopo):
    tipo = tipo.lower()
    dimensoes = ''.join(['[%s]'%v.split('..')[-1] for v in dimensoes.split(',')])
    var_dict(tipos_variaveis[tipo])
    t_vars[tipos_variaveis[tipo]] += ["%s%s"%(v.strip().lower(),dimensoes) for v in nomes_var.split(',')]
    
  
  for tipo in t_vars:
    for nome in t_vars[tipo]:
      idx = nome.find('[')
      if(idx > 0):
        n_vars[nome[:idx]] = tipo
      else:
        n_vars[nome] = tipo
  
  return t_vars,n_vars

###Classe principal

In [498]:
class VisualG2C():
  def __init__(self, programa):
    self.__stack = []
    self.__programa = limpa_comentario(programa)
    self.t_vars,self.n_vars = variaveis_globais(self.__programa)

  def __parse_bloco(self, lin):
    lin = lin.strip()
    ret = None
    tokens = (
        lin
        .replace('(', ' ( ')
        .replace(')', ' ) ')
        .split(' ')
    )
    try:
      ret = eval("parse_%s(lin, self.n_vars, self.t_vars)"%tokens[0].lower())
    except NameError:
      ret = None
    nv = len(self.__stack)
    for token in tokens:
      token = token.lower()
      if(token in palavras_chave_fechamento):
        if(palavras_chave[self.__stack[-1]] == token):
          self.__stack.pop()
          nv -= 1
        else:
          print(palavras_chave[self.__stack[-1]])
          raise Exception("Erro no delimitador <%s>"%token)
      if(token in palavras_chave):
        self.__stack.append(token)
    
    
    if(not ret and lin in palavras_chave_fechamento):
      ret = '}'
    return nv,ret
      

  def eval(self):
    idx = self.__programa.find('inicio')
    if(idx < 0):
      raise Exception("Programa não possui <inicio>!!!")

    for valor in apelidos:
      self.__programa = self.__programa.replace(valor," %s "%apelidos[valor])

    programa_C = [""]
    
    for lin in self.__programa[idx:].split('\n'):
      nv, p = self.__parse_bloco(lin)
      programa_C.append('\t'*(nv))
      if(p):
        programa_C[-1] += (p)
      else:
        programa_C[-1] += (lin)
      
      

    if(len(self.__stack)):
      raise Exception("Erro de sintaxe em <%s>!!!"%self.__stacK[-1])

    for linha in programa_C:
      print(linha) 
    programa_C = '\n'.join(programa_C)

    return programa_C



#Exemplo

In [499]:
p = VisualG2C(programa)

In [500]:
with open(nome_arq_c, 'w') as f:
  f.write(p.eval())

"{0}" "{0}"
nome nome
"{0}" "{0}"
i i
"{1}" "{1}"
"{0}" "{0}"
max max
"{0}" "{0}"
min min
"{0}" "{0}"
soma soma
"{0}" "{0}"
media media


#include <stdio.h>
#include <stdbool.h>
#include <string.h>

int main() {
	char nome[500];
	int max, min, i, soma, numeros[5];
	float media, teste1[3][4], teste2[3][4], teste3[3][4], teste4[3][4];
	memset(&nome, 0, sizeof(nome));
	memset(&max, 0, sizeof(max));
	memset(&min, 0, sizeof(min));
	memset(&i, 0, sizeof(i));
	memset(&soma, 0, sizeof(soma));
	memset(&numeros, 0, sizeof(numeros));
	memset(&media, 0, sizeof(media));
	memset(&teste1, 0, sizeof(teste1));
	memset(&teste2, 0, sizeof(teste2));
	memset(&teste3, 0, sizeof(teste3));
	memset(&teste4, 0, sizeof(teste4));
	
	printf("Digite seu nome: ");
	scanf("%s", nome);
	
	max = numeros[1];
	for(i = 1; i <= 5; i++) {
		printf("%s, digite o %do valor: ", nome, i);
		scanf("%d", &numeros[i]);
		
		    soma  =  soma + numeros[i];
		    media =  soma / 5;
		
		if ((numeros[i] > max) ) {
			    max =  numer

In [501]:
!gcc $nome_arq_c -o teste
!./teste

Digite seu nome: Guilherme
Guilherme, digite o 1o valor: 1
Guilherme, digite o 2o valor: 2
Guilherme, digite o 3o valor: 3
Guilherme, digite o 4o valor: 4
Guilherme, digite o 5o valor: 5
maior numero: 5
menor numero: 1
Soma dos numeros digitados:15
Média dos numero digitados:3.000000